In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel,GPT2Tokenizer,TrainingArguments,Trainer,TextDataset,DataCollatorForLanguageModeling

In [ ]:
model_name='gpt2'
tokenizer=GPT2Tokenizer.from_pretrained(model_name)
model=GPT2LMHeadModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [ ]:
data=pd.read_csv('/content/chat.csv')

In [ ]:
data.shape

(10016, 1)

In [ ]:
data.head()

,Answers
0,Hello there. Tell me how are you feeling today?
1,Hi there. What brings you here today?
2,Hi there. How are you feeling today?
3,Great to see you. How do you feel currently?
4,Hello there. Glad to see you. What is going on...


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10016 entries, 0 to 10015
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Answers  10016 non-null  object
dtypes: object(1)
memory usage: 78.4+ KB


In [ ]:
def load_dataset(file_path,tokenizer,block_size=1024):
    train_data=TextDataset(tokenizer=tokenizer,file_path=file_path,block_size=block_size)
    return train_data

In [ ]:
def load_data_collator(tokenizer,mlm=False):
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm=mlm)
    return data_collator

In [ ]:
def train(train_file_path,model_name,output_dir,overwrite_output_dir,train_batch_size,epochs):
    tokenizer=GPT2Tokenizer.from_pretrained(model_name)
    train_data=load_dataset(train_file_path,tokenizer)

    data_collator=load_data_collator(tokenizer)

    tokenizer.save_pretrained(output_dir)

    model=GPT2LMHeadModel.from_pretrained(model_name)
    model.save_pretrained(output_dir)

    training_arguments=TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=overwrite_output_dir,
        num_train_epochs=epochs,
        per_device_train_batch_size=train_batch_size,
        logging_dir='./log',
        logging_steps=100,
        save_steps=150,
        logging_first_step=True,
        save_total_limit=2,
        learning_rate=0.001,
      )

    trainer=Trainer(
        model=model,
        args=training_arguments,
        data_collator=data_collator,
        train_dataset=train_data
    )
    hist=trainer.train()
    trainer.save_model()
    return trainer,hist

In [ ]:
train_file_path='/content/chat.csv'
model_name=model_name
output_dir='./model'
overwrite_output_dir=True
train_batch_size=3
epochs=17

In [ ]:
hist=train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    train_batch_size=train_batch_size,
    epochs=epochs)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running

Step,Training Loss
1,2.892200
100,3.457600
200,3.043500
300,2.943000
400,2.357800
500,2.228200
600,2.253400
700,2.126400
800,1.625600
900,1.678800


Saving model checkpoint to ./model/checkpoint-150
Configuration saved in ./model/checkpoint-150/config.json
Model weights saved in ./model/checkpoint-150/pytorch_model.bin
Saving model checkpoint to ./model/checkpoint-300
Configuration saved in ./model/checkpoint-300/config.json
Model weights saved in ./model/checkpoint-300/pytorch_model.bin
Saving model checkpoint to ./model/checkpoint-450
Configuration saved in ./model/checkpoint-450/config.json
Model weights saved in ./model/checkpoint-450/pytorch_model.bin
Deleting older checkpoint [model/checkpoint-150] due to args.save_total_limit
Saving model checkpoint to ./model/checkpoint-600
Configuration saved in ./model/checkpoint-600/config.json
Model weights saved in ./model/checkpoint-600/pytorch_model.bin
Deleting older checkpoint [model/checkpoint-300] due to args.save_total_limit
Saving model checkpoint to ./model/checkpoint-750
Configuration saved in ./model/checkpoint-750/config.json
Model weights saved in ./model/checkpoint-750/py